# Setup

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'C:\Users\utente\Documents\GitHub\Project2'
sys.path.insert(0,my_path + r'/code/Flavio')

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
print('TensorFlow imported')

import helpers as dh

# imports
import numpy as np 
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense

print('Keras imported')

# fix random seed for reproducibility
np.random.seed(666)

TensorFlow imported


Using TensorFlow backend.


Keras imported


# Preparing the data

### Importing dictionary

###### Remember to import the stanford dataset found in https://nlp.stanford.edu/projects/glove/

In [2]:

import os
os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_stanford')

glove_dict = dh.build_glove_dict('glove.twitter.27B.100d.txt')

print('glove_dict length = ' ,len(glove_dict))

print('Features of \',\' : \n', glove_dict[','])


glove_dict length =  1193514
Features of ',' : 
 [  3.22990000e-01  -5.82930000e-01  -2.33270000e-01   3.44090000e-01
  -5.69340000e-01   1.32110000e+00  -8.62670000e-02   2.05260000e-01
   5.83200000e-01   4.75160000e-01   4.57290000e-01  -2.60950000e-01
  -2.95560000e+00   1.67340000e-01   4.65400000e-01   1.95300000e-03
   3.11760000e-01  -4.92880000e-01  -4.95870000e-01  -2.89710000e-01
   2.05190000e-01  -9.54020000e-02  -5.54190000e-01   2.57660000e-01
  -5.55870000e-01  -2.47970000e+00  -5.64550000e-01  -1.79400000e-01
   4.22600000e-01  -3.22630000e-01  -3.37600000e-01   4.13700000e-01
  -2.61110000e-01   5.80890000e-01  -2.13750000e-01   1.29900000e-01
  -1.00040000e-02  -1.43990000e-01  -1.83220000e-01  -3.06110000e-01
  -1.44840000e+00   7.22220000e-01  -4.70500000e-01   4.09470000e-01
   1.69630000e-01  -1.45480000e-01   1.04160000e-01   5.42010000e-02
  -5.80360000e-01  -4.55060000e-01  -2.84750000e-01  -2.75170000e-02
   1.06780000e-01  -1.04710000e-01  -6.76610000e-02  -

### Extracting tweets from preprocessed dataset

###### Remember to build the dataset with the preprocessing procedure in PRE_generate_txt_preproc

In [3]:
import os
#os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_epfl\short')
os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_epfl\full')

tweets = dh.build_tweets_matrix('all_full_processed.txt')
        
print('tweets lenght: ', len(tweets))

print('tweets sample (tweets[0]) :', tweets[1])

tweets lenght:  2280482
tweets sample (tweets[0]) : ['because' 'your' 'logic' 'is' 'so' 'dumb' ',' 'i' 'wo' 'not' 'even' 'crop'
 'out' 'your' 'name' 'or' 'your' 'photo' '.' 'tsk' '.' '<url>']


### Build sentence features: averages+min+max

In [2]:
'''
The script will produce a series of .txt files in order to progressively save the splitted X matrix on the hard drive. 
This was done because the RAM of my PC is not big enough to contain the dictionary, the X matrix and the tweets at the same time.
It also deletes the tweets that aren't useful anymore.
'''

'\nThis script will produce a series of .txt files in order to save the splitted X matrix on the hard drive, this was done because \nthe RAM of my PC is not big enough to contain the dictionary, the X matrix and the tweets at the same time\n\n'

###### Parameters: set feature_size and delete_window

In [4]:
import progressbar
import gc

tweetlen = len(tweets)

bar = progressbar.ProgressBar(maxval=tweetlen, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

feat_size = 100

delete_win = 300000

os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_epfl\Xmatrix')

###### Building training dataset

In [5]:
X = np.empty([delete_win, feat_size*3])

bar.start()

k = 0
i = 0

while i < tweetlen:
    
    #everytime it enters in this if it creates a file for the X matrix to free RAM memory
    if ((i%delete_win == 0) & (i!=0)):
        tweets = np.delete(tweets, np.arange(0,delete_win) ,0)
        strname = 'X'+ str(k) +'.txt'
        np.savetxt(strname,X, delimiter=',')
        k = k+1
        X = np.empty([delete_win, feat_size*3])
    
    feats = []
    for j in range(len(tweets[i-k*delete_win])):
        if tweets[i-k*delete_win][j] in glove_dict:
            feats.append(glove_dict[tweets[i-k*delete_win][j]])
        else:
            feats.append([0]*feat_size)

    X[i-k*delete_win]= np.hstack((np.mean(feats, axis=0), np.amax(feats, axis=0), np.amin(feats, axis=0)))
        
    i = i+1
    
    bar.update(i)
    
bar.finish()
    
print('X shape: ' , X.shape)
print('X[9][5] = ', X[9][5], ' ', type(X[9][5]))
print('X[9] = ', X[9])



[========================================================================] 100%


X shape:  (300000, 300)
X[9][5] =  0.210989071429   <class 'numpy.float64'>
X[9] =  [ -3.23717643e-02  -6.47349643e-02   1.15616393e-01   6.34630107e-02
  -7.29200714e-02   2.10989071e-01   7.12279643e-02  -1.52967857e-03
   3.06660714e-02   4.91523571e-02  -5.25741786e-02  -8.16685714e-02
  -3.12509179e+00  -8.01772857e-02  -6.99717500e-02  -1.28014184e-01
   9.45028214e-02   5.05464286e-02   5.91748214e-02  -1.62991000e-01
  -2.15696511e-01  -5.04910357e-02   1.15846496e-01   1.45981821e-01
   1.86174143e-01  -6.54297357e-01  -9.52389286e-02   3.67039286e-03
   5.35619393e-01   1.15220560e-01  -2.72081643e-01  -1.42230250e-01
  -4.28881536e-01   2.30295539e-01   2.83070857e-01   1.01007036e-01
   1.83310000e-02  -2.74448929e-03  -2.08975357e-02   3.20697357e-02
  -8.95353250e-01   1.85881286e-01  -2.08809429e-01   1.87128429e-01
   4.46815607e-01  -1.55061779e-01   9.36145714e-02   1.77959393e-01
  -1.11808571e-01  -7.22662393e-02  -2.56615893e-01  -2.87647536e-01
   8.78699286e-02  

###### Label generation

In [ ]:
labels = []

halfway = 1127644 #For the full dataset
#halfway = 90233    #For the short dataset

for i in range(halfway):
    labels.append(1)
for i in range(len(tweets)-halfway):
    labels.append(0)

Y = np.asarray(labels)

###### Deleting dictionary and tweets to free RAM

In [6]:
del(tweets)
del(glove_dict)
del(labels)

# Neural Network

###### Sample matrix rebuilt from files

In [ ]:
X = np.empty([0,feat_size*3])

#total number of files produced:
final_num = 6

for i in range (0,final_num+1):
    print(i)
    strf = 'X' + str(i) +'.txt'
    X = np.concatenate((X,np.loadtxt(strf, delimiter = ',')))
    
print(X.shape)

0
1
2
3
4
5


###### Definition of the network params

In [ ]:
l_input = feat_size*3
l1 = feat_size*3
l2 = feat_size*3
l3 = feat_size
l_output = 1

###### Model

In [ ]:
model = Sequential()
model.add(Dense(l1, input_dim=l_input, activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros'))
model.add(Dense(l2, activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros'))
model.add(Dense(l3, activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros'))
model.add(Dense(l_output, activation='sigmoid', kernel_initializer='glorot_uniform', bias_initializer='zeros'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

###### Training

In [ ]:
model.fit(X, Y, epochs=50, batch_size=250, verbose=2)

# Testing

###### Importing dictionary and testing set (as before)

In [ ]:
import os
os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_stanford')

glove_dict = dh.build_glove_dict('glove.twitter.27B.100d.txt')

print('glove_dict length = ' ,len(glove_dict))

print('Features of \',\' : \n', glove_dict[','])


In [ ]:
import os
#os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_epfl\short')
os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_epfl')

test_tweets = dh.build_tweets_matrix('test_data.txt')
        
print('tweets lenght: ', len(test_tweets))

print('tweets sample (tweets[0]) :', test_tweets[1])

###### Build sentence features: averages+min+max

In [ ]:
import progressbar

bar = progressbar.ProgressBar(maxval=len(test_tweets), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

feat_size = 100


X = np.empty([len(test_tweets), feat_size*3])

bar.start()

for i in range(len(test_tweets)):
    feats = []
    for j in range(len(test_tweets[i])):
        if test_tweets[i][j] in glove_dict:
            feats.append(glove_dict[test_tweets[i][j]])
        else:
            feats.append([0]*feat_size)

    X[i]= np.hstack((np.mean(feats, axis=0), np.amax(feats, axis=0), np.amin(feats, axis=0)))

    bar.update(i)
    
bar.finish()
    
print('X shape: ' , X.shape)
print('X[9][5] = ', X[9][5], ' ', type(X[9][5]))
print('X[9] = ', X[9])




In [ ]:
del(glove_dict)
del(test_tweets)


###### Predictions

In [ ]:
predictions = model.predict(X, verbose=1)

###### Labelling predictions

In [ ]:
print(len(predictions))

for i in range(len(predictions)):
    if predictions[i] < 0.5:
        predictions[i] = -1
    else:
        predictions[i] = 1

print(predictions)



###### Create submission

In [ ]:
from create_csv_submission import create_csv_submission

ids = range(1,10001)

name = 'subNN100'
create_csv_submission(ids, predictions, name)

In [ ]:
print(ids)